# Anti-Fraud Prototype (ML + NLP)

Цель проекта — продемонстрировать базовый антифрод-пайплайн:
генерация и анализ данных,
feature engineering,
обучение МЛ-модели,
пример NLP.

Данные ненастоящие и используются для демонстрации подхода.

In [1]:
# 1. Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Генерация данных

Используется ненастоящий датасет для бинарной классификации,
имитирующий антифрод-задачу (фрод / нефрод).

In [2]:
# 2. Загрузка данных
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_classes=2,
    random_state=42
)

df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['is_fraud'] = y

## 2. EDA

Быстрый обзор данных:
первые строки датасета
 распределение целевой переменной
базовая статистика признаков

In [7]:
df.head()
df['is_fraud'].value_counts
df.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,is_fraud
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.480472,-0.011035,-0.072376,-0.492447,0.469059,0.499845,0.455850,0.017115,-0.058077,0.631419,0.503000
std,1.583809,1.618548,1.024196,1.518933,1.489777,1.621358,2.000162,1.029048,1.046402,2.806143,0.500241
min,-4.661168,-5.814203,-3.031194,-4.258034,-3.889284,-4.341477,-7.298063,-3.254479,-3.582063,-8.102614,0.000000
25%,-0.542215,-1.132212,-0.779532,-1.504082,-0.609955,-0.674893,-0.776099,-0.676648,-0.744499,-1.034635,0.000000
50%,0.583361,0.029747,-0.041891,-0.760929,0.541561,0.449055,0.553223,0.025772,-0.081367,0.987519,1.000000
75%,1.521656,1.077206,0.644444,0.230285,1.447875,1.531734,1.757939,0.679153,0.659029,2.371964,1.000000
max,5.066061,4.605669,3.276399,5.685693,6.036793,5.608412,7.456970,3.089890,2.986329,7.933944,1.000000


## 3. Пример SQL запроса для антифрода

Пример запроса, который может использоваться для расчёта
поведенческих признаков в системе.

SELECT user_id, 
       COUNT(*) as transactions_last_hour,
       AVG(amount) as avg_amount,
       LAG(amount) OVER(PARTITION BY user_id ORDER BY time) as prev_amount
FROM transactions
WHERE time > NOW() - INTERVAL '1 hour'
GROUP BY user_id;

## 4. Feature Engineering

Создание дополнительных признаков на основе исходных данных:
сумма признаков
отношения признаков.

In [ ]:
# 5. Feature engineering

# Сумма первых 5 фич
df['feature_sum'] = df.iloc[:, :5].sum(axis=1)
# Отношение двух фич
df['feature_ratio'] = df['feature_0'] / (df['feature_1'] + 0.001)

## 5. ML модель

Задействована логистическая регрессия как базовая модель для антифрод-задачи.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1487
)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83       158
           1       0.80      0.86      0.83       142

    accuracy                           0.83       300
   macro avg       0.83      0.83      0.83       300
weighted avg       0.83      0.83      0.83       300



## 6. NLP компонент

Пример обработки текстовых данных с помощью корзин слов.
Тексты рассматриваются как дополнительный сигнал.

In [ ]:
texts = [
    "urgent transfer needed",
    "normal transaction",
    "suspicious activity detected"
]

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(texts)

vectorizer.get_feature_names_out()

## Выводы и ограничения

Проект показывает базовый антифрод пайплайн.
Используются ненастоящие данные.
Модель и NLP-компонент представлены в упрощённом виде.

Проект служит демонстрацией подхода.